### Bad Poets Society – Poem Generator (Python)

This notebook documents the development of a small poem generator for the Bad Poets Society assignment.

We use three themed inspiring sets (love, nature, dream), extract word lists with spaCy,
and then recombine them with simple templates and randomness. 

In [ ]:
# Loading libraries
import spacy
from collections import Counter
from pathlib import Path

nlp = spacy.load("en_core_web_sm")


In [ ]:
# load the inspiring sets
from pathlib import Path

love_text = Path("poems_love.txt").read_text(encoding="utf-8")
nature_text = Path("poems_nature.txt").read_text(encoding="utf-8")
dream_text = Path("poems_dream.txt").read_text(encoding="utf-8")

#### Extracting themed vocabularies with spacy

Here I use spaCy to process each inspiring set and extract three types of words:

- nouns,
- adjectives,
- verbs.

I keep lemmas instead of surface forms, and filter out very short or non-alphabetic
tokens. This gives me three separate vocabularies (love, nature, dream) that define
the style and imagery of each theme.


In [20]:
def extract_vocab(text: str):
    doc = nlp(text)

    nouns = set()
    adjs = set()
    verbs = set()

    for token in doc:
        if not token.is_alpha:
            continue
        lemma = token.lemma_.lower()

        # remove very short words
        if len(lemma) <= 2:
            continue

        if token.pos_ == "NOUN":
            nouns.add(lemma)
        elif token.pos_ == "ADJ":
            adjs.add(lemma)
        elif token.pos_ == "VERB":
            verbs.add(lemma)

    return sorted(nouns), sorted(adjs), sorted(verbs)

#### Small grammar fixes with inflection

Early tests showed many subject–verb agreement errors (e.g. “someone love the tree”)
and only singular nouns. In this part I use:

- `lemminflect` to choose a 3rd person singular verb form (e.g. `someone loves`),
- `inflect` to create plural nouns when the template asks for `N_PL`.

The goal is not perfect grammar, but to reduce the most obvious errors while
keeping the “bad poetry” flavour.


In [ ]:
from lemminflect import getInflection
import inflect

# Initialize inflect engine for noun pluralization
inflect_engine = inflect.engine()

def inflect_verb(lemma: str, pronoun: str | None) -> str:
    """
    Do a small grammar fix on the verb form based on the subject pronoun:
    - for he / she / it / someone / no one → use 3rd person singular (VBZ)
    - for other cases (I / you / we / they / None) → keep the base lemma
    """
    if pronoun is None:
        return lemma

    lower = pronoun.lower()

    if lower in ["he", "she", "it", "someone", "no one"]:
        infl = getInflection(lemma, tag="VBZ")
        return infl[0] if infl else lemma
    return lemma


def pluralize_noun(noun: str) -> str:
    """
    Use inflect to turn a noun into its plural form.
    For example: child → children, sky → skies.
    If the conversion fails, fall back to the original word.
    """
    plural = inflect_engine.plural(noun)
    return plural if plural else noun


In [22]:
NOUNS_LOVE, ADJ_LOVE, VERBS_LOVE = extract_vocab(love_text)
NOUNS_NATURE, ADJ_NATURE, VERBS_NATURE = extract_vocab(nature_text)
NOUNS_DREAM, ADJ_DREAM, VERBS_DREAM = extract_vocab(dream_text)

#### Shared function words

In addition to theme-specific vocabularies, I also define some shared word lists:

- adverbs,
- pronouns,
- time expressions.

These are used across all themes and help give the lines a bit more rhythm and
coherence 


In [23]:
ADVERBS = [
    "slowly", "almost", "again", "barely", "softly", "quietly", "tonight",
    "always", "forever", "never", "suddenly", "secretly", "somehow", "gently",
    "briefly", "carefully"
]

PRONOUNS = ["I", "you", "we", "no one", "someone"]

TIMES = [
    "at midnight", "before dawn", "after the last train", "on a Tuesday",
    "at closing time", "at 3 a.m.", "before the storm", "after work"
]

#### Cleaning the vocabularies

Some words from the inspiring sets lead to outputs that are either too distracting or
simply not useful for my generator.  
In this cell I apply small manual blacklists to remove a few nouns and verbs.  
This is a simple but important design choice: I am already shaping what my “bad poet”
is allowed to talk about.


In [24]:
# filtering bad words
BAD_LOVE_NOUNS = {"crime", "bank"}  
NOUNS_LOVE = [w for w in NOUNS_LOVE if w not in BAD_LOVE_NOUNS]

BAD_NATURE_NOUNS = {"flesh"}
NOUNS_NATURE = [w for w in NOUNS_NATURE if w not in BAD_NATURE_NOUNS]


#### Word pools per theme

This section bundles everything into a single structure: `WORD_POOLS_THEMED`.

For each theme (`"love"`, `"nature"`, `"dream"`), I store the corresponding nouns,
adjectives and verbs, plus the shared adverbs, pronouns and times.  
The rest of the generator can now work just by switching the `theme` parameter, while
reusing the same code.


In [25]:
WORD_POOLS_THEMED = {
    "love": {
        "N": NOUNS_LOVE,
        "ADJ": ADJ_LOVE,
        "V": VERBS_LOVE,
        "ADV": ADVERBS,
        "PRON": PRONOUNS,
        "T": TIMES,
    },
    "nature": {
        "N": NOUNS_NATURE,
        "ADJ": ADJ_NATURE,
        "V": VERBS_NATURE,
        "ADV": ADVERBS,
        "PRON": PRONOUNS,
        "T": TIMES,
    },
    "dream": {
        "N": NOUNS_DREAM,
        "ADJ": ADJ_DREAM,
        "V": VERBS_DREAM,
        "ADV": ADVERBS,
        "PRON": PRONOUNS,
        "T": TIMES,
    },
}


#### Line templates

Here I define a small set of line templates. 

In [26]:
LINE_PATTERNS = [
    ["PRON", "V", "the", "ADJ", "N"],          # I touch the silent night
    ["in", "the", "ADJ", "N"],                # in the bright field
    ["beneath", "the", "ADJ", "N"],           # beneath the yellow breeze
    ["sometimes", "PRON", "V", "the", "N"],   # sometimes we hold the dream
    ["PRON", "V", "N", "T"],                  # I follow dream at midnight
    ["in", "the", "ADJ", "N", "PRON", "V", "N", "T"], 
    ["the", "ADJ", "N", "V", "ADV"],          # the subtle road fade slowly
]


In [ ]:
import random

def generate_line(theme: str = "love") -> str:
    """
    Generate a single line of poetry for a given theme.

    Logic:
    - Randomly select a pattern from LINE_PATTERNS.
    - Lowercase tokens (in, the, beneath, ...) are used as literal words.
    - Uppercase tokens are interpreted as placeholders:
        - PRON: sample a pronoun from the theme's PRON list and remember it
        - V:    sample a verb lemma from V, then inflect it based on the current pronoun
        - N:    sample a singular noun
        - N_PL: sample a noun and convert it to its plural form using inflect
    """
    if theme not in WORD_POOLS_THEMED:
        raise ValueError(
            f"Unknown theme: {theme}. Available themes: {list(WORD_POOLS_THEMED.keys())}"
        )

    pools = WORD_POOLS_THEMED[theme]
    pattern = random.choice(LINE_PATTERNS)

    words: list[str] = []
    current_pronoun: str | None = None

    for token in pattern:
        if token.islower():
            # Fixed lowercase word
            words.append(token)
        elif token == "PRON":
            pron = random.choice(pools["PRON"])
            current_pronoun = pron
            words.append(pron)
        elif token == "V":
            verb_lemma = random.choice(pools["V"])
            verb_form = inflect_verb(verb_lemma, current_pronoun)
            words.append(verb_form)
        elif token == "N_PL":
            noun_lemma = random.choice(pools["N"])
            noun_form = pluralize_noun(noun_lemma)
            words.append(noun_form)
        elif token == "N":
            noun_lemma = random.choice(pools["N"])
            words.append(noun_lemma)
        else:
            # Other tags, such as ADJ, ADV, T (time expressions)
            words.append(random.choice(pools[token]))

    line = " ".join(words)
    return line[0].upper() + line[1:]


In [28]:
def generate_title(theme: str = "love") -> str:
    pools = WORD_POOLS_THEMED[theme]
    adj = random.choice(pools["ADJ"]).capitalize()
    noun = random.choice(pools["N"])
    return f"{adj} {noun}"


In [29]:
def generate_poem(
    theme: str = "love",
    min_lines: int = 6,
    max_lines: int = 10,
    with_title: bool = True,
    seed: int | None = None,
) -> str:
    if theme not in WORD_POOLS_THEMED:
        raise ValueError(f"Unknown theme: {theme}. Use 'love' or 'nature'.")

    if seed is not None:
        random.seed(seed)

    num_lines = random.randint(min_lines, max_lines)
    lines = [generate_line(theme=theme) for _ in range(num_lines)]

    if with_title:
        title = generate_title(theme=theme)
        poem_str = title + "\n" + "-" * len(title) + "\n" + "\n".join(lines)
    else:
        poem_str = "\n".join(lines)

    return poem_str


In [30]:
print("=== DREAM THEME ===\n")
for i in range(10):
    print(generate_poem(theme="dream"))

=== DREAM THEME ===

Happy nightmare
---------------
In the true heart
The dangerous house continue forever
In the cloudy dust
Sometimes no one fatigues the leisure
We subdue willow after work
Someone prevails the hard scenery
In the fresh breaks someone shocks space before dawn
The clear grave hate slowly
You achieve scalp after the last train
Right end
---------
Beneath the good spring
Sometimes we waste the pond
You collide melody at midnight
You process the unfamiliar tree
You walk the well darkness
The smart angel come carefully
Someone steps the wonderful opportunity
Very desire
-----------
The firm creation wound barely
Sometimes you bless the bed
In the unreal harbinger
Sometimes you reign the infinity
The blank fulfilled perceive slowly
No one freezes vacuum at 3 a.m.
Ready kid
---------
Beneath the well mockup
In the hungry clock
Sometimes no one bleeds the grave
Sometimes we survive the suffering
Sometimes no one bears the ecstasy
Sometimes someone rises the focus
In the dri

In [31]:
print("=== LOVE THEME ===\n")
for i in range(10):
    print(generate_poem(theme="love"))

print("\n=== NATURE THEME ===\n")
for i in range(10):
    print(generate_poem(theme="nature"))


=== LOVE THEME ===

Lonely dinosaur
---------------
In the cheerful grief
Sometimes someone pays the spear
In the mindful really
In the deafening system no one instructs list at 3 a.m.
The menacing shield समझ almost
In the beloved king
Sometimes you speak the spirit
I swell home on a Tuesday
The tough mai dance always
Beneath the careless fray
Permanent well
--------------
In the barren thunder no one needs girl after the last train
In the real coating
In the about song we practice compliment after work
In the lonely reply you widen shareholder before the storm
Beneath the easy cry
In the unknown happiness
The valuable dong come briefly
Sometimes you base the madness
Someone requires the strange hunger
Inevitable truce
----------------
You send arm on a Tuesday
In the only abbie
You thrive the amusing cry
You suppress the charming thing
Someone admits the regretful stage
I hide work before the storm
In the mere business we light ranger before dawn
In the heavenly breeze
Beneath the ind

#### Reflecting on creativity

In this project, my poem generator combines themed vocabularies (love, nature, dream),
a small set of line templates, controlled randomness, and light grammatical inflection.
To think about the creativity of such a system, I focused on three aspects:

- **Novelty** – whether the system keeps producing new and surprising combinations of
  words across different runs.
- **Thematic consistency** – whether poems generated under different themes actually
  differ in mood, imagery and atmosphere.
- **Aesthetic interest** – whether some lines or images feel evocative, ironic or
  memorable enough to be worth keeping or presenting.

My evaluation was qualitative rather than numerical. I generated many poems, read
through them, and manually selected a small number of favourites for each theme
(these are recorded in `the3_for_ai.txt`). These selected poems were then used as
the basis for image and audio generation, which often amplified their perceived
meaning and emotional tone.